In [13]:
import random
import spacy
import math
from datetime import date
import webbrowser
from pathlib import Path
from PIL import Image
from pytesseract import pytesseract
import json


# Configuration variables
g_keywords = ["chinese", "chinaman", "chink"]
g_search_date_range_start = date(1850, 1, 1)
g_search_date_range_end = date(2024, 7, 9)
g_num_articles_per_time_period = 50
g_time_block_range_years = 5

g_search_date_range_days = (g_search_date_range_end - g_search_date_range_start).days
g_num_time_blocks = math.ceil(g_search_date_range_days / (365 * g_time_block_range_years))

nlp = spacy.load("en_core_web_trf")


def get_time_block_number(year, search_date_range_start, time_block_range_years):
    return math.floor((year - search_date_range_start.year) / time_block_range_years)


def get_starting_date(time_block_num, search_date_range_start, time_block_range_years):
    return date(search_date_range_start.year + time_block_num * time_block_range_years, 1, 1)


def get_ending_date(time_block_num, search_date_range_start, search_date_range_end, time_block_range_years):
    ending_date = date(search_date_range_start.year + (time_block_num + 1) * time_block_range_years - 1, 12, 31)
    if ending_date > search_date_range_end:
        return search_date_range_end
    else:
        return ending_date


def get_keywords_with_num_hits(keywords, start_date, end_date):
    keywords_with_num_hits = {}
    for keyword in keywords:
        webbrowser.open("https://www.newspapers.com/search/results/?country=us&date-end=" + end_date.strftime('%Y-%m-%d') + "&date-start=" + start_date.strftime('%Y-%m-%d') + "&entity-types=page&keyword=" + keyword)
        keywords_with_num_hits[keyword] = int(input("Number of hits: ").replace(",", ""))
    return keywords_with_num_hits


def get_keywords_with_num_articles(keywords_with_num_hits, num_articles_per_time_period):
    total_hits = sum(keywords_with_num_hits.values())
    article_remainders = []
    keywords_with_num_articles = {}
    for keyword, hits in keywords_with_num_hits.items():
        keywords_with_num_articles[keyword] = int((hits / total_hits) * num_articles_per_time_period)
        article_remainders.append((keyword, ((hits / total_hits) * num_articles_per_time_period) % 1))
    article_remainders.sort(key=lambda keyword_with_article_remainder: keyword_with_article_remainder[1], reverse=True)
    num_searches_short = num_articles_per_time_period - sum(keywords_with_num_articles.values())
    for i in range(num_searches_short):
        keywords_with_num_articles[article_remainders[i][0]] += 1
    return keywords_with_num_articles


def get_keywords_with_urls_and_num_articles(keywords_with_num_articles, start_date, end_date):
    keywords_with_urls_and_num_articles = []
    for keyword, num_articles in keywords_with_num_articles.items():
        keywords_with_urls_and_num_articles.append({
            "keyword": keyword,
            "url": "https://www.newspapers.com/search/results/?country=us&date-end=" + end_date.strftime('%Y-%m-%d') + "&date-start=" + start_date.strftime('%Y-%m-%d') + "&entity-types=page&keyword=" + keyword,
            "num_articles": num_articles
        })
    return keywords_with_urls_and_num_articles


def run_article_download_helper(keywords_with_urls_and_num_articles):
    for keyword_with_urls_and_num_articles in keywords_with_urls_and_num_articles:
        text_input = input("Please download " + str(keyword_with_urls_and_num_articles["num_articles"]) + " articles with the keyword \"" + keyword_with_urls_and_num_articles["keyword"] + "\". Press return to continue. Type \"stop\" to stop.")
        if text_input == "stop":
            break
        webbrowser.open(keyword_with_urls_and_num_articles["url"])
        text_input = input("Press return to continue. Type \"stop\" to stop.")
        if text_input == "stop":
            break


def get_contexts_for_keyword_from_text(text, keyword):
    contexts = []
    sentences = list(nlp(text).sents)
    for num in range(len(sentences) - 2):
        sentence = sentences[num + 1]
        if keyword in sentence.text.lower():
            contexts.append(sentences[num].text + " " + sentence.text + " " + sentences[num + 2].text)
    return contexts            
            
      
def get_all_contexts_for_keywords(articles_dir_path):
    keywords_with_contexts = {}
    for articles_by_keyword_dir_path in articles_dir_path.rglob("*"):
        if articles_by_keyword_dir_path.is_dir():
            keyword = articles_by_keyword_dir_path.name
            keywords_with_contexts[keyword] = []
            for article_path in articles_by_keyword_dir_path:
                text = pytesseract.image_to_string(Image.open(article_path))
                date_components = article_path.stem.split("-")
                date_of_text = date(int(date_components[0]), int(date_components[1]), int(date_components[2]))
                keywords_with_contexts[keyword].append({
                    'date': date_of_text,
                    'contexts': get_contexts_for_keyword_from_text(text, keyword)
                })
    return keywords_with_contexts


def narrow_contexts(contexts_for_keywords):
    narrowed_contexts_per_keyword = {}
    for keyword, all_dates_and_contexts in contexts_for_keywords.items():
        narrowed_contexts_per_keyword[keyword] = []
        for date_and_article_contexts in all_dates_and_contexts:
            while True:
                if len(date_and_article_contexts) == 0:
                    narrowed_contexts_per_keyword[keyword].append({
                        'date': date_and_article_contexts["date"],
                        'context': "Placeholder"})
                    break
                else:
                    random_index = random.randint(0, len(date_and_article_contexts) - 1)
                    article_context = date_and_article_contexts['contexts'][random_index]
                    print(article_context)
                    should_use_context = input("Use this context? (y/n or stop)")
                    if should_use_context == "y":
                        narrowed_contexts_per_keyword[keyword].append({
                            'date': date_and_article_contexts["date"],
                            'context': article_context})
                        return None
                    elif should_use_context == "stop":
                        break
                    else:
                        date_and_article_contexts['contexts'].pop(random_index)
    return narrowed_contexts_per_keyword


def write_contexts_to_file(contexts, time_block_number):
    if g_narrowed_contexts_per_keyword is not None:
        with open('contexts/time_period_' + g_time_block_number + "_contexts.json", 'w+') as contexts_file:
            json.dump(g_narrowed_contexts_per_keyword, contexts_file)
    

print("Number of time blocks: " + str(g_num_time_blocks))
print("Number of articles to download: " + str(g_num_time_blocks * g_num_articles_per_time_period))

g_time_block_number = int(input("Pick time block, 0-" + str(g_num_time_blocks - 1) + ": "))
if g_time_block_number < 0 or g_time_block_number >= g_num_time_blocks:
    print("Invalid time block!")
    exit()

g_start_date = get_starting_date(g_time_block_number, g_search_date_range_start, g_time_block_range_years)
g_end_date = get_ending_date(g_time_block_number, g_search_date_range_start, g_search_date_range_end, g_time_block_range_years)

print("Start date: " + g_start_date.strftime('%Y-%m-%d'))
print("End date: " + g_end_date.strftime('%Y-%m-%d'))
print("Length of time period in days: " + str((g_end_date - g_start_date).days))

Number of time blocks: 35
Number of articles to download: 1750
Start date: 1850-01-01
End date: 1854-12-31
Length of time period in days: 1825


Get the number of articles needed for each keyword by pasting in the number of hits for each keyword

In [14]:
g_keywords_with_num_articles = get_keywords_with_num_articles(get_keywords_with_num_hits(g_keywords, g_start_date, g_end_date), g_num_articles_per_time_period)
print(g_keywords_with_num_articles)

ValueError: invalid literal for int() with base 10: ''

Generate the URLs to grab the articles from

In [11]:
g_keywords_with_urls_and_num_articles = get_keywords_with_urls_and_num_articles(g_keywords_with_num_articles, g_start_date, g_end_date)
print("Generated " + str(len(g_keywords_with_urls_and_num_articles)) + " URLs")

Generated 3 URLs


Utility to guide users in downloading articles. Place articles in a folder with the same name as the keyword, which will be in the "articles" folder. Name the file with the date in the format "YYYY-MM-DD"

In [12]:
run_article_download_helper(g_keywords_with_urls_and_num_articles)

Get contexts surrounding keywords. Example value for g_contexts_for_keywords:
{ 'chinese': [{ 'date': date(2021, 7, 9), 'contexts': ["Sentence 1", "Sentence 2"]}], 'japanese': [{ 'date': date(2019, 2, 23), 'contexts': ["Sentence 3", "Sentence 4"]}] 

In [ ]:
g_articles_dir_path = Path.cwd() / 'articles'
g_contexts_for_keywords = get_all_contexts_for_keywords(g_articles_dir_path)

Narrow down contexts to only on per article. Contexts are presented to the user in a random order, and the first one that the user confirms is relevant is used. If none are confirmed, the context is replaced with a placeholder. Results are stored in a JSON file that should be manually edited to fix placeholders before moving on to analysis.

In [ ]:
g_narrowed_contexts_per_keyword = narrow_contexts(g_contexts_for_keywords)
write_contexts_to_file(g_narrowed_contexts_per_keyword, g_time_block_number)